# Simple Cryptocurrency Dashboard

## Import Libraries

In [116]:
# Import Libraries
import os
from dotenv import load_dotenv

import requests
import pandas as pd
import panel as pn
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from datetime import datetime, timedelta

## Extension

In [119]:
pn.extension('plotly')

## Styles

### Panel Style

In [4]:
# App Style
ACCENT = "teal"

styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

### Colors for Plotly

In [126]:
# Colors
colors_a = {
    'BTC': 'orange',
    'ETH': 'mediumpurple',
    'BNB': 'indianred'
}

colors_b = {
    'BTC': 'gold',
    'ETH': 'plum',
    'BNB': 'lightsalmon'
}

## Data Extraction & Transformation

### Binance API

In [130]:
# Load API Key from .env file
load_dotenv('keys.env')
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')

# Binance API Constants
BINANCE_API_URL = 'https://api.binance.us/api/v3/klines'
BINANCE_API_URL_CURRENT_PRICE = 'https://api.binance.us/api/v3/ticker/price?symbol='

### Helper Functions

#### Fetch Historical Data

In [134]:
@pn.cache()
def fetch_historical_data(symbol='BTCUSDT', interval='1d', start_time=None, end_time=None, limit=1000):
    """Fetch historical data for a given symbol from Binance API."""
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit,
        'startTime': start_time,
        'endTime': end_time
    }
    response = requests.get(BINANCE_API_URL, headers={'X-MBX-APIKEY': BINANCE_API_KEY}, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    data = response.json()
    df = pd.DataFrame(data, columns=[
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close Time', 'Quote Asset Volume', 'Number of Trades',
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ])
    df['Date'] = pd.to_datetime(df['Open Time'], unit='ms')
    df = df.drop(columns=['Open Time', 'Close Time', 'Quote Asset Volume',
                          'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
    df['Symbol'] = symbol[:-4]
    df['Close'] = df['Close'].astype(float)
    return df

#### Fetch Current Price

In [137]:
@pn.cache()
def fetch_current_price(symbol):
    """Fetch the current price for a given symbol from Binance API."""
    url = BINANCE_API_URL_CURRENT_PRICE + symbol
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return float(data.get('price', 0))
    else:
        print(f"Error fetching current price for {symbol}: {response.status_code}")
        return None

#### Add Moving Averages

In [140]:
def add_moving_averages(df):
    """Add Simple and Exponential Moving Averages to the DataFrame."""
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    return df

#### Convert Colors for Plotly

In [143]:
def convert_color(color_name, opacity=0.8):
    """Convert a color name to rgba format."""
    rgba = mcolors.to_rgba(color_name, opacity)
    return f'rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]})'

### Time Intervals

In [146]:
# Intervals
intervals = {
    'All_Time': [None, None],
    '5Y': [int((datetime.now() - timedelta(days=5*365)).timestamp() * 1000), None],
    '1Y': [int((datetime.now() - timedelta(days=365)).timestamp() * 1000), None],
    '6M': [int((datetime.now() - timedelta(days=180)).timestamp() * 1000), None],
    '3M': [int((datetime.now() - timedelta(days=90)).timestamp() * 1000), None],
    '1M': [int((datetime.now() - timedelta(days=30)).timestamp() * 1000), None],
    '2W': [int((datetime.now() - timedelta(days=14)).timestamp() * 1000), None],
    '1W': [int((datetime.now() - timedelta(days=7)).timestamp() * 1000), None],
}

### Data Extraction  & Transformation II

In [149]:
# Fetching Data and Building DataFrames

symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
dataframes = []
ath_dict = {}
current_price_dict = {}

# Populate dataframes and dictionaries
for symbol in symbols:
    df = fetch_historical_data(symbol=symbol)
    df = add_moving_averages(df)
    dataframes.append(df)
    
    ath_dict[symbol[:-4]] = df['High'].max()
    current_price_dict[symbol[:-4]] = fetch_current_price(symbol)

combined_df = pd.concat(dataframes)

## Widgets

In [152]:
# Dashboard Components

symbol_selector = pn.widgets.Select(name='Cryptocurrency', options=[symbol[:-4] for symbol in symbols], value='BTC')
interval_selector = pn.widgets.Select(name='Time Interval', options=list(intervals.keys()), value='1Y')

## Figures

In [179]:
@pn.cache()
@pn.depends(symbol_selector)
def plot_ath_comparison(symbol):
    # Fetch data
    df_symbol = combined_df[combined_df['Symbol'] == symbol]
    ath = ath_dict.get(symbol, 0)
    current_price = current_price_dict.get(symbol, 0)

    # Check if values are floats; otherwise, convert or set to 0
    try:
        ath = float(ath)
    except ValueError:
        ath = 0

    try:
        current_price = float(current_price)
    except ValueError:
        current_price = 0

    # Create plots
    fig_ath = go.Figure()

    fig_ath.add_trace(go.Bar(
        name=f'{symbol} All-Time-High: $ {ath:.2f}',
        y=[symbol],
        x=[ath],
        marker_color=convert_color(colors_b[symbol], 0.6),
        orientation='h',
        hovertemplate=f'{symbol}<br>All-Time-High: $ %{{x:.2f}}<extra></extra>'
    ))

    fig_ath.add_trace(go.Bar(
        name=f'{symbol} Current Price: $ {current_price:.2f}',
        y=[symbol],
        x=[current_price],
        marker_color=convert_color(colors_a[symbol], 0.8),
        orientation='h',
        hovertemplate=f'{symbol}<br>Latest Price: $ %{{x:.2f}}<extra></extra>'
    ))

    fig_ath.update_layout(
        title_text=f"{symbol} Market Overview: Current Price vs. All-Time High",
        xaxis_title="Price (USD)",
        barmode='overlay',
        xaxis_type='log',
        template='plotly_white'
    )
    return pn.pane.Plotly(fig_ath, height=350)

In [169]:
plot_ath_comparison('BTC')

Plotly(Figure, height=350)

In [181]:
@pn.cache()
@pn.depends(symbol_selector, interval_selector)
def plot_price_over_time(symbol, interval):
    # Fetch data
    df_symbol = combined_df[combined_df['Symbol'] == symbol]
    ath = ath_dict.get(symbol, 0)
    current_price = current_price_dict.get(symbol, 0)

    # Check if values are floats; otherwise, convert or set to 0
    try:
        ath = float(ath)
    except ValueError:
        ath = 0

    try:
        current_price = float(current_price)
    except ValueError:
        current_price = 0

    # Create plots
    fig_price = go.Figure()

    # Filter data by interval
    start_time, end_time = intervals[interval]
    df_filtered = df_symbol[(df_symbol['Date'] >= pd.to_datetime(start_time, unit='ms'))] if start_time else df_symbol

    fig_price.add_trace(go.Scatter(
        x=df_filtered['Date'],
        y=df_filtered['Close'],
        mode='lines',
        name=f'{symbol} Close',
        line=dict(color=convert_color(colors_a[symbol], 0.8)),
        fill='tozeroy',
        fillcolor=convert_color(colors_b[symbol], 0.6),
        hovertemplate=f'{symbol}<br>Date: %{{x}}<br>Close: $ %{{y:.2f}}<extra></extra>'
    ))

    fig_price.update_layout(
        title_text=f"{symbol} Price Over Time ({interval})",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        yaxis_type="log",
        #xaxis_rangeslider_visible=True,
        template='plotly_white'
    )

    return pn.pane.Plotly(fig_price, height=500)

In [183]:
plot_price_over_time('BTC', '1Y')

Plotly(Figure, height=500)

## App

In [189]:
pn.template.FastListTemplate(
    title="Cryptocurrency Dashboard",
    sidebar=[symbol_selector, interval_selector],
    main=[pn.Column(plot_ath_comparison, plot_price_over_time, sizing_mode="stretch_both")],
    main_layout=None,
    accent=ACCENT,
).servable();

# Simple Cryptocurrency Dashboard [Script-Version]

**How2Use**
- Uncomment content of following code block, copy&paste and save as .py script.
- Serve App in terminal with ``bokeh serve crypto_prices_over_time.py``  

In [177]:
# # Cryptocurrency Dashboard

# # Import Libraries
# import os
# from dotenv import load_dotenv

# import requests
# import pandas as pd
# import panel as pn
# import plotly.graph_objects as go
# import matplotlib.colors as mcolors
# from datetime import datetime, timedelta

# pn.extension('plotly')

# # Load API Key from .env file
# load_dotenv('keys.env')
# BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')

# # Binance API Constants
# BINANCE_API_URL = 'https://api.binance.us/api/v3/klines'
# BINANCE_API_URL_CURRENT_PRICE = 'https://api.binance.us/api/v3/ticker/price?symbol='

# # Helper Functions

# @pn.cache()
# def fetch_historical_data(symbol='BTCUSDT', interval='1d', start_time=None, end_time=None, limit=1000):
#     """Fetch historical data for a given symbol from Binance API."""
#     params = {
#         'symbol': symbol,
#         'interval': interval,
#         'limit': limit,
#         'startTime': start_time,
#         'endTime': end_time
#     }
#     response = requests.get(BINANCE_API_URL, headers={'X-MBX-APIKEY': BINANCE_API_KEY}, params=params)

#     if response.status_code != 200:
#         print(f"Error {response.status_code}: {response.text}")
#         return None

#     data = response.json()
#     df = pd.DataFrame(data, columns=[
#         'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
#         'Close Time', 'Quote Asset Volume', 'Number of Trades',
#         'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
#     ])
#     df['Date'] = pd.to_datetime(df['Open Time'], unit='ms')
#     df = df.drop(columns=['Open Time', 'Close Time', 'Quote Asset Volume',
#                           'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
#     df['Symbol'] = symbol[:-4]
#     df['Close'] = df['Close'].astype(float)
#     return df

# @pn.cache()
# def fetch_current_price(symbol):
#     """Fetch the current price for a given symbol from Binance API."""
#     url = BINANCE_API_URL_CURRENT_PRICE + symbol
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         return float(data.get('price', 0))
#     else:
#         print(f"Error fetching current price for {symbol}: {response.status_code}")
#         return None

# def add_moving_averages(df):
#     """Add Simple and Exponential Moving Averages to the DataFrame."""
#     df['SMA_50'] = df['Close'].rolling(window=50).mean()
#     df['SMA_200'] = df['Close'].rolling(window=200).mean()
#     df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
#     df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
#     return df

# def convert_color(color_name, opacity=0.8):
#     """Convert a color name to rgba format."""
#     rgba = mcolors.to_rgba(color_name, opacity)
#     return f'rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]})'

# # Colors
# colors_a = {
#     'BTC': 'orange',
#     'ETH': 'mediumpurple',
#     'BNB': 'indianred'
# }

# colors_b = {
#     'BTC': 'gold',
#     'ETH': 'plum',
#     'BNB': 'lightsalmon'
# }

# # Intervals
# intervals = {
#     'All_Time': [None, None],
#     '5Y': [int((datetime.now() - timedelta(days=5*365)).timestamp() * 1000), None],
#     '1Y': [int((datetime.now() - timedelta(days=365)).timestamp() * 1000), None],
#     '6M': [int((datetime.now() - timedelta(days=180)).timestamp() * 1000), None],
#     '3M': [int((datetime.now() - timedelta(days=90)).timestamp() * 1000), None],
#     '1M': [int((datetime.now() - timedelta(days=30)).timestamp() * 1000), None],
#     '2W': [int((datetime.now() - timedelta(days=14)).timestamp() * 1000), None],
#     '1W': [int((datetime.now() - timedelta(days=7)).timestamp() * 1000), None],
# }

# # Fetching Data and Building DataFrames

# symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
# dataframes = []
# ath_dict = {}
# current_price_dict = {}

# # Populate dataframes and dictionaries
# for symbol in symbols:
#     df = fetch_historical_data(symbol=symbol)
#     df = add_moving_averages(df)
#     dataframes.append(df)
    
#     ath_dict[symbol[:-4]] = df['High'].max()
#     current_price_dict[symbol[:-4]] = fetch_current_price(symbol)

# combined_df = pd.concat(dataframes)

# # Dashboard Components

# symbol_selector = pn.widgets.Select(name='Cryptocurrency', options=[symbol[:-4] for symbol in symbols], value='BTC')
# interval_selector = pn.widgets.Select(name='Time Interval', options=list(intervals.keys()), value='1Y')

# @pn.depends(symbol_selector, interval_selector)
# def update_dashboard(symbol, interval):
#     # Fetch data
#     df_symbol = combined_df[combined_df['Symbol'] == symbol]
#     ath = ath_dict.get(symbol, 0)
#     current_price = current_price_dict.get(symbol, 0)

#     # Check if values are floats; otherwise, convert or set to 0
#     try:
#         ath = float(ath)
#     except ValueError:
#         ath = 0

#     try:
#         current_price = float(current_price)
#     except ValueError:
#         current_price = 0

#     # Create plots
#     fig_ath = go.Figure()

#     fig_ath.add_trace(go.Bar(
#         name=f'{symbol} All-Time-High: $ {ath:.2f}',
#         y=[symbol],
#         x=[ath],
#         marker_color=convert_color(colors_b[symbol], 0.6),
#         orientation='h',
#         hovertemplate=f'{symbol}<br>All-Time-High: $ %{{x:.2f}}<extra></extra>'
#     ))

#     fig_ath.add_trace(go.Bar(
#         name=f'{symbol} Current Price: $ {current_price:.2f}',
#         y=[symbol],
#         x=[current_price],
#         marker_color=convert_color(colors_a[symbol], 0.8),
#         orientation='h',
#         hovertemplate=f'{symbol}<br>Latest Price: $ %{{x:.2f}}<extra></extra>'
#     ))

#     fig_ath.update_layout(
#         title_text=f"{symbol} Market Overview: Current Price vs. All-Time High",
#         xaxis_title="Price (USD)",
#         barmode='overlay',
#         xaxis_type='log',
#         template='plotly_white'
#     )

#     # Generate interval price curve
#     fig_price = go.Figure()

#     # Filter data by interval
#     start_time, end_time = intervals[interval]
#     df_filtered = df_symbol[(df_symbol['Date'] >= pd.to_datetime(start_time, unit='ms'))] if start_time else df_symbol

#     fig_price.add_trace(go.Scatter(
#         x=df_filtered['Date'],
#         y=df_filtered['Close'],
#         mode='lines',
#         name=f'{symbol} Close',
#         line=dict(color=convert_color(colors_a[symbol], 0.8)),
#         fill='tozeroy',
#         fillcolor=convert_color(colors_b[symbol], 0.6),
#         hovertemplate=f'{symbol}<br>Date: %{{x}}<br>Close: $ %{{y:.2f}}<extra></extra>'
#     ))

#     fig_price.update_layout(
#         title_text=f"{symbol} Price Over Time ({interval})",
#         xaxis_title="Date",
#         yaxis_title="Price (USD)",
#         yaxis_type="log",
#         #xaxis_rangeslider_visible=True,
#         template='plotly_white'
#     )

#     return pn.Column(
#         pn.pane.Markdown(f"## {symbol} Overview"),
#         pn.pane.Plotly(fig_ath, height=350),
#         pn.pane.Plotly(fig_price, height=500)
#     )
    
# # Layout the dashboard
# dashboard = pn.Column(
#     "# Cryptocurrency Dashboard",
#     symbol_selector,
#     interval_selector,
#     update_dashboard
# )

# # Serve the dashboard
# dashboard.servable()

## Troubleshooting

In [8]:
import requests

# Replace <your-ip-address> with the actual IP address
url = "http://your_ip/5006/crypto_prices_over_time" # find IP in terminal: hostname -I
response = requests.get(url)

# Check the response
print(response.status_code)
print(response.text)

403
<HTML>
  <HEAD>
     Access Denied
  </HEAD>
<BODY>

<h1>Access Denied</h1>

<p>
Access to arbitrary websites is not available from free accounts;
you can only access sites that are on our
<a href="http://www.pythonanywhere.com/whitelist">allowlist</a>.
If you want to suggest something to add to our whitelist
drop us a line at PythonAnywhere Support <nbusercare@pythonanywhere.com>.  It will have
to have an official public API.
</p>


<p>
Alternatively, you can sign up for a paid account at
<a href="http://www.pythonanywhere.com/account/">http://www.pythonanywhere.com/account/</a>
</p>
<p>
If you have already got a paid account and you're still getting this messge,
you may need to reload your web app (from the "Web" tab) or restart
your consoles.  If that doesn't help, drop us a line at PythonAnywhere Support <nbusercare@pythonanywhere.com>.
</p>

</BODY>

